# Final Project | How to predict a pandemic 
by Kevin Spurk

Text

# Table of content

1. 
2.
3.
...

# 1 | Setup

### 1.1. Libary imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('max_row', None)

### 1.2 Data imports

### 1.2.1 google mobility data

In [19]:
# csv import
data_mob_global = pd.read_csv('data/mobility_google/Global_Mobility_Report.csv')

# dict of countries and there languages to search in 
countries_lang = {'GB': 'en', 'DE': 'de', 'FR': 'fr', 'NL': 'nl',
                  'ES': 'es', 'PT': 'pt', 'PL': 'pl', 'IT': 'it',
                  'AT': 'de', 'DK': 'da', 'SE': 'sv', 'GR': 'el',  'CZ': 'cs'}

# getting subset of row with data of the target countries
tgt_countries = [country_code for country_code in countries_lang.keys()]
data_mob_eu = data_mob_global[data_mob_global['country_region_code'].isin(tgt_countries)]

len(data_mob_eu)


1363509

### 1.2.2 facebook mobility data

### 1.2.3 google search trends data

In [3]:
# connect to google and deepL APIs 
from pytrends.request import TrendReq
from translate import Translator 

# get authentification for deepL API
with open('auth/deepl_auth_key.txt') as f:
    deepl_auth_key = f.read()

# google trends API access request
pytrends = TrendReq(hl='en-US', tz=0)

In [8]:
# keyword list 
keywords_list = ['covid', 'corona', 'SARS-CoV-2', 'virus', 'symptoms', 'fever', 'cough', 'tiredness', 'loss of smell', 'throat', 'shortness of breath', 'breathing issues', 'headache', 'chest pain', 'corona test', 'covid test', 'test', 'sick', 'infection']

# translating keyword list into target languages
from pyf.apis import deepl_translate_list

kw_dict, kw_df = deepl_translate_list(auth=deepl_auth_key, keywords=keywords_list, lang_out=countries_lang)


In [8]:
kw_df

,GB,DE,FR,NL,ES,PT,PL,IT,AT,DK,SE,GR,CZ
0,covid,covid,covide,covid,covid,covid,covid,covid,covid,covid,covid,covid,covid
1,corona,corona,corona,corona,corona,corona,korona,corona,corona,corona,corona,corona,corona
2,SARS-CoV-2,SARS-CoV-2,SRAS-CoV-2,SARS-CoV-2,SARS-CoV-2,SRA-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
3,virus,Virus,virus,virus,virus,vírus,wirus,virus,Virus,virus,virus,ιός,virus
4,symptoms,Symptome,symptômes,symptomen,síntomas,sintomas,objawy,sintomi,Symptome,symptomer,symtom,συμπτώματα,příznaky
5,fever,Fieber,fièvre,koorts,fiebre,febre,gorączka,febbre,Fieber,feber,feber,πυρετός,horečka
6,cough,Husten,toux,hoest,toser,tosse,kaszel,tosse,Husten,hoste,hosta,βήχας,kašel
7,tiredness,Müdigkeit,fatigue,vermoeidheid,cansancio,cansaço,zmęczenie,stanchezza,Müdigkeit,træthed,trötthet,κούραση,únava
8,loss of smell,Geruchsverlust,la perte de l'odorat,verlies van reukzin,pérdida del olfato,perda do olfacto,utrata węchu,perdita dell'olfatto,Geruchsverlust,tab af lugt,förlust av lukt,απώλεια οσμής,ztráta čichu
9,throat,Kehle,gorge,keel,garganta,garganta,gardło,gola,Kehle,hals,hals,λαιμός,krk


In [29]:
import time

kw_list = ['cold']

# build payload
pytrends.build_payload(kw_list, cat=0, timeframe='2020-01-02 2021-11-02', geo='GB')


In [32]:
data4 = pytrends.get_historical_interest(kw_list, year_start=2020, month_start=1, day_start=2, hour_start=0, year_end=2021, month_end=11, day_end=2, hour_end=23, sleep=60)

In [34]:
data4.tail()

,cold,isPartial
date,,
2021-11-02 19:00:00,58,False
2021-11-02 20:00:00,62,False
2021-11-02 21:00:00,61,False
2021-11-02 22:00:00,65,False
2021-11-02 23:00:00,74,False


In [37]:
data4.reset_index(inplace=True)

In [50]:
data4.tail(30)

,date,cold,isPartial
14479,2021-11-01,54,False
14480,2021-11-01,54,False
14481,2021-11-01,59,False
14482,2021-11-01,59,False
14483,2021-11-01,66,False
14484,2021-11-01,73,False
14485,2021-11-02,71,False
14486,2021-11-02,79,False
14487,2021-11-02,90,False
14488,2021-11-02,96,False


In [57]:
data4 = data4.drop(['isPartial'], axis=1)

In [44]:
data4['date'] = data4['date'].dt.date

In [52]:
data4 = data4.groupby(['date']).mean()

In [55]:
data4['country'] = 'GB'

In [58]:
data4.head(20)

,cold,country
date,,
2020-01-02,65.041667,GB
2020-01-03,66.041667,GB
2020-01-04,69.250000,GB
2020-01-05,68.416667,GB
2020-01-06,65.083333,GB
2020-01-07,64.791667,GB
2020-01-08,58.708333,GB
2020-01-09,28.600000,GB
2020-01-10,27.791667,GB


In [ ]:
data_empty = pd.DataFrame({})

In [30]:
data5 = pytrends.related_queries()

In [31]:
data5

{'cold': {'top':                      query  value
  0                 cold war    100
  1                cold sore     31
  2            cold symptoms     24
  3                cold call     21
  4                cold feet     20
  5             call of duty     18
  6    cold war call of duty     17
  7             hot and cold     14
  8               cold sores     14
  9             the cold war     13
  10      black ops cold war     11
  11            cod cold war     11
  12               baby cold     11
  13    cold weather payment     11
  14            feeling cold     10
  15             common cold      9
  16        cold war zombies      9
  17              stone cold      9
  18              cold hands      8
  19            cold and flu      8
  20     cold symptoms covid      7
  21               head cold      7
  22          covid symptoms      7
  23  how to get rid of cold      6
  24             cold shower      6,
  'rising':                                  que

In [ ]:
data3 = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
data3

In [ ]:
data6 = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
data6

In [ ]:
#1 Interest over Time
'''
data = pytrends.interest_over_time() 
data = data.reset_index() 

#1 Interest over Time plot

import plotly.express as px

fig = px.line(data, x="date", y=['covid'], title='Keyword Web Search Interest Over Time')
fig.show() '''

In [ ]:
len(data)#.head()

In [ ]:
data